## Cosmological constraints with BAO
I provide you with a series of power spectrum monopole measurements, at effective redshifts $z_{\mathrm{zeff}}$ = 0.1 to 1.9.
In this session, you will:

1) measure the BAO isotropic parameter $\alpha(z_{\mathrm{eff}}) = \left[D_{V}(z_{\mathrm{eff}})/r_{\mathrm{drag}}\right] / \left[D_{V}^{\mathrm{fid}}(z_{\mathrm{eff}})/r_{\mathrm{drag}}^{\mathrm{fid}}\right]$ with each power spectrum

2) with these measurements, you will constrain $\Omega_{m}, \Omega_{k}$ (hence detect dark energy!)

3) provided a value of $r_{\mathrm{drag}}$ (in $\mathrm{Mpc}$), you will perform an "inverse distance ladder" analysis: fit all the $\alpha(z_{\mathrm{eff}})$ by varying $H_{0}$ and $\Omega_{m}$

4) those finding the correct $\Omega_{m}$, $\Omega_{k}$ and $H_{0}$ (I know the truth, you don't!) win!

5) do you remember how to compute the correlation function? (last year's TD) What about the power spectrum? If any question, please ask!

### Installation

#### Packages
**If packages are already installed, skip this part.**

In [ ]:
# On Google Colab: I have already prepared the environment,
# let's download it.
# This may be faster than pip install below.
#!python -m pip install gdown
#!gdown https://drive.google.com/uc?id=1IKWH9vW5Zh910K7FZ4bbrNZaejvD9_Ru
#!unzip -oq TD_clustering_venv.zip
#import sys
#sys.path.append('./TD_clustering_venv/lib/python3.10/site-packages')

In [ ]:
# Alternatively:
!python -m pip install matplotlib cython emcee corner camb
!python -m pip install git+https://github.com/cosmodesi/cosmoprimo#egg=cosmoprimo[camb,astropy]

#### Note
Within the DESI collaboration we have put some effort into setting up some ~ easy to use Python packages for standard clustering analyses. Check them out here: https://github.com/cosmodesi.
You may e.g. be interested by:
- [cosmoprimo](https://github.com/cosmodesi/cosmoprimo): primordial cosmology (class, camb, isitgr, fftlog, interpolator, BAO filtering)
- [pycorr](https://github.com/cosmodesi/pycorr): correlation function estimation
- [pypower](https://github.com/cosmodesi/pypower): power spectrum (and window function) estimation
- [pyrecon](https://github.com/cosmodesi/pyrecon): standard BAO reconstruction
- [mockfactory](https://github.com/cosmodesi/mockfactory): tools to be build fast mocks
- [desilike](https://github.com/cosmodesi/desilike): DESI likelihoods, fits of 2-pt statistics, Fisher, bindings with cosmological samplers (Cobaya, CosmoSIS, MontePython)

#### Measurements
I generated fake (Gaussian-distributed) power spectrum measurements, in a fake cosmology; these are in the "sims" folder of the repository.

In [ ]:
!git clone https://github.com/adematti/TD_clustering.git

In [ ]:
import sys
sys.path.insert(0, 'TD_clustering')  # add to PYTHONPATH

### A look at power spectrum measurements

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from environment import Measurement, list_path_sim_measurement  # specify paths

list_path = list_path_sim_measurement()
list_data = []

for path in list_path:
    m = Measurement.load(path)
    print('At effective redshift {:.2f} first k are {}, pk0 {}'.format(m.attrs['zeff'], m.x[:2], m.y[0,:2]))
    plt.plot(m.x, m.x * m.y[0], label='$z = {:.2f}$'.format(m.attrs['zeff']))
    list_data.append(m)

plt.xlabel('$k$ [$h/\mathrm{Mpc}$]')
plt.ylabel('$k P(k)$ [$(\mathrm{Mpc}/h)^{2}$]')
plt.legend()
plt.show()
# m is an instance of Measurement
# k (wavenumber) array is m.x
# pk (power spectrum) is m.y
# m.y is of shape (1, len(m.x)): the first Legendre multipole (monopole).
print('Attributes: {}'.format(m.attrs))

### Isotropic BAO power spectrum model

The principle of BAO fits is to measure *only* the position of the BAO, marginalizing over the shape of the power spectrum, for this measurement to be robust to distortions due to observational systematics.
For this we decompose the fiducial power spectrum into *wiggle* and *no-wiggle* parts. We will allow the *wiggle* part to move, keeping the *no-wiggle* part fixed, adding $k$-dependent nuisance terms (*broadband terms*) to adjust the shape of the data power spectrum.

In [ ]:
# First generate a linear power spectrum in $(\mathrm{Mpc}/h)^{3}$ units; z = 0 is enough.
# (Indeed, linear growth factor D^{2}(z) is degenerate with B^2 in the model below)
# Fiducial cosmological parameters are:
# dict(Omega_m=0.31, Omega_b=0.022/0.676**2, h=0.676, sigma8=0.8, n_s=0.97, m_ncdm=0.06)
# Several options:
# 1) use cosmoprimo (engine='class' or engine='camb'), see https://github.com/cosmodesi/cosmoprimo/blob/main/nb/examples.ipynb
#from cosmoprimo import Cosmology
#cosmo_fid = Cosmology(Omega_m=0.31, omega_b=0.022, h=0.676, sigma8=0.8, n_s=0.97, m_ncdm=[0.06], engine='camb')
#klin = np.geomspace(1e-4, 5., 2000)
#pk_interpolator = cosmo_fid.get_fourier().pk_interpolator().to_1d(z=0.)
#pklin = pk_interpolator(klin)
#np.savetxt('pklin.txt', np.column_stack([klin, pklin]))
# 2) use classy, camb...
# 3) load precomputed linear power spectrum:
#klin, pklin = np.loadtxt('pklin.txt', unpack=True)

In [1]:
# Then you need a smooth power spectrum, matching the linear power spectrum but without BAO wiggles.
# Several options:
# 1) use cosmoprimo
#from cosmoprimo import PowerSpectrumInterpolator1D
#pk_interpolator = PowerSpectrumInterpolator1D(klin, pklin, extrap_kmin=1e-4, extrap_kmax=1e2)
#from cosmoprimo import PowerSpectrumBAOFilter
#pknow = PowerSpectrumBAOFilter(pk_interpolator, engine='wallish2018').smooth_pk_interpolator()(klin)
# 2) fit the linear power spectrum in log/log with a polynomial (Hinton2017); not perfect but good enough for this notebook
#logk = np.log(klin)
#logpk = np.log(pklin)
#maxk = logk[pklin.argmax()]
#gauss = np.exp(-0.5 * ((logk - maxk) / 1.)**2)
#w = np.ones_like(klin) - 0.5 * gauss
#series = np.polynomial.polynomial.Polynomial.fit(logk, logpk, deg=13 ,w=w)
#pknow = np.exp(series(logk))

In [2]:
# Plot wiggles = linear power spectrum / no-wiggle power spectrum

Model equation is (https://arxiv.org/abs/1705.06373, but many other formulations exist):
$P(k,\alpha) = B^{2} [P_{\mathrm{nw}}(k) + \sum_{i=0}^{2} A_{i} k^{i}] \lbrace 1 + [\mathcal{O}(k/\alpha) - 1] e^{- \frac{1}{2} \Sigma_{\mathrm{nl}}^{2}k^{2}} \rbrace$  
$P_{\mathrm{nw}}$ is the no-wiggle power spectrum.  
$\mathcal{O}(k) = P_{\mathrm{lin}}(k)/P_{\mathrm{nw}}(k)$ are the wiggles. 
Free parameters are: $\alpha, B, A_{i}$, parameter of cosmological interest is $\alpha$ (which shifts BAO wiggles).
Choose for the non-linear damping of BAO wiggles $\Sigma_{\mathrm{nl}} = 6.7\;\mathrm{Mpc/h}$.
See https://arxiv.org/abs/2404.03000 and https://arxiv.org/abs/2402.14070 for the BAO theory used in DESI, including a new broadband parameterization.

In [ ]:
sigmanl = 6.7

def pk_iso(k, alpha, B, A0, A1, A2):
    """Code here the BAO power spectrum model."""
    ipknow = np.interp(k, klin, pknow)
    poly = A0 + A1 * k + A2 * k**2
    damped_wiggles = "code wiggles shifted by alpha here"
    return B**2 * (ipknow + poly) * damped_wiggles

### Model fitting
Repeat, for each redshift z:  
- build $\chi^{2} = (\mathrm{data} - \mathrm{model})^T C^{-1} (\mathrm{data} - \mathrm{model})$
- minimize chi2 for example with scipy.optimize.minimize, and check the fit looks good by plotting data and best fit model

In [ ]:
from scipy import optimize

nbb = 3
init = [1., 2.] + [0.] * nbb  # alpha, B, broadbands
bounds = [(0.88, 1.12), (1., 5.)] + [(-5e4, 5e4)] * nbb

for m in list_data:
    covariance = m.cov
    invcovariance = np.linalg.inv(covariance)
    k = m.x
    data = m.y[0]
    
    def chi2(args):
        """Write chi2, args is alpha, B, A0, A1, A2."""
        model = pk_iso(k, *args)  # return model
        diff = data - model
        toret = "write chi2 here"
        return toret

    """Check reduced chi2, make plot of data, model, error bars."""
    #result = optimize.minimize(chi2, init, method='SLSQP', bounds=bounds)
    #args = result.x  # best fit chi2
    #minchi2 = "chi2 at best fit"
    #msg = 'Reduced chi2 is {:.4f}/({:d} - {:d}) = {:.4f}'.format(minchi2, len(k), len(args), minchi2 / (len(k) - len(args)))
    #print(msg)
    """Make plot of data, model, error bars."""
    #kplt = klin[(klin > k[0]) & (klin < k[-1])]
    #plt.errorbar(k, k * data, k * np.diag(covariance) ** 0.5, linestyle='none', color='C0')
    #plt.plot(kplt, kplt * pk_iso(kplt, *args), label='model', color='C0')
    #plt.show()

We want the PDF of $\alpha$. Let's define the posterior and sample it with e.g. https://github.com/dfm/emcee. If you are not familiar with Bayesian inference, you can take a quick look at the corresponding notebook.
$\alpha$ posterior is well approximated by a Gaussian, so let's just take its mean and covariance for the downstream cosmological inference.

In [ ]:
import emcee   # pip install --user emcee
import corner  # pip install --user corner

nsteps = 4000
ndim = len(bounds)
nwalkers = 2 * ndim
list_alpha_mean, list_alpha_covariance = [], []
rng = np.random.RandomState(seed=42)

for m in list_data:  # loop over all z's

    covariance = m.cov
    invcovariance = np.linalg.inv(covariance)
    k = m.x
    data = m.y[0]
    
    def chi2(args):
        """Write chi2, args is alpha, B, A0, A1, A2."""
    
    def logprior(args):
        """
        Write flat uniform prior between bounds, args is alpha, B, A0, A1, A2.
        WARNING: This is the *log*-prior, so...
        0 (or constant) if bounds[i][0] < args[i] < bounds[i][1]
        -inf else.
        """
    
    # Write posterior here (take flat priors between bounds on all parameters)
    def logposterior(args):
        """
        Write log-posterior, args is alpha, B, A0, A1, A2.
        WARNING: This is the *log*-posterior.
        """
    
    #sampler = emcee.EnsembleSampler(nwalkers, ndim, logposterior)
    #start = [[rng.normal(v, (b[1] - b[0]) / 10.) for v, b in zip(init, bounds)] for i in range(nwalkers)]
    #sampler.run_mcmc(start, nsteps)
    #samples = sampler.get_chain(flat=True)
    # Look at samples, remove burnin
    #samples = samples[len(samples) // 2:]
    # Take mean (or median) and covariance of samples
    #mean = np.mean(samples[:, 0])
    #covariance = np.cov(samples[:, 0])
    #list_alpha_mean.append(mean)
    #list_alpha_covariance.append(covariance)
    #labels = ['alpha', 'B'] + ['A{:d}'.format(i) for i in range(nbb)]
    #fig = corner.corner(samples, labels=labels, quantiles=[0.16, 0.5, 0.84], show_titles=True, title_kwargs={'fontsize': 14})
    #plt.show()
    #plt.close(fig)

### Expansion history
The measured $\alpha$ can be modelled by $\alpha_{\mathrm{theory}}(z) = \left[D_{V}(z_{\mathrm{eff}})/r_{\mathrm{drag}}\right] / \left[D_{V}^{\mathrm{fid}}(z_{\mathrm{eff}})/r_{\mathrm{drag}}^{\mathrm{fid}}\right]$ (why?)  
$D_{V}(z) = z^{1/3} D_{M}(z)^{2/3} D_{H}(z)^{1/3}$ with $D_{M}(z)$ comoving transverse distance, $D_{H}(z) = c/H(z)$ "Hubble distance". Factor $z^{1/3}$ is purely conventional.

Code up $D_{M}(z)$, $D_{H}(z)$ as a function of $\Omega_{m}$ and $\Omega_{k}$ (faster than creating a new cosmology at each MCMC step).  
Build the $\chi^{2}$ of the previously measured $\alpha$, using $\alpha_{\mathrm{theory}}$ as a model.
Previously measured $\alpha$ `list_alpha_mean` should be considered independent (zero cross-correlations); hence the covariance matrix is diagonal, with `list_alpha_covariance` as diagonal elements.
This $\chi^{2}$ depends on $r_{\mathrm{drag}}$ (in $\mathrm{Mpc}/h$), $\Omega_{m}$ and $\Omega_{k}$ if free curvature (entering $D_{M}$ and $D_{H}$).

In [ ]:
list_z, list_dv_over_rs_fid = [], []

for data in list_data:
    z = data.attrs['zeff']
    #list_z.append(z)
    # Precompute DV_fid / rs_drag_fid
    #dv = z**(1. / 3.) * ((1 + z) * cosmo_fid.angular_diameter_distance(z))**(2. / 3.) * (100. * cosmo_fid.efunc(z))**(-1. / 3.) /
    #list_dv_over_rs_fid.append(dv)

list_z = np.array(list_z)
list_dv_over_rs_fid = np.array(list_dv_over_rs_fid)

# Define Hubble parameter, comoving transverse distance, spherically-averaged distance DV
# model, function of (rs_drag, Omega_m, Omega_k), is DVf / rs_drag / (DV_fid / rs_drag_fid) at each data z.
# You can use cosmoprimo, but I recommend that you recode this up from the math, the inference will run faster.
# If running on Google Colab, you may ask the AI! (but be careful ;))

def spherically_averaged_distance(z, Omega_m=0.3, Omega_k=0.):
    # return spherically averaged distance at z for given Omega_m and Omega_k

def list_alpha_model(rs, *args):
    return np.array([spherically_averaged_distance(z, *args) for z in list_z]) / rs / list_dv_over_rs_fid


Use a very wide prior on $r_{\mathrm{drag}}$ as we don't want to make any assumption about the primordial Universe: we only assume constant comoving $r_{\mathrm{drag}}$ ("standard ruler") to constrain expansion history.
Sample the posterior and draw the contours $\Omega_{m} - \Omega_{k}$. See? we detect dark energy! (and zero curvature)

In [ ]:
def chi2(args):
    """Write chi2, assuming independent measurements at each redshift."""
    toret = 0.
    for data_alpha, cov, model_alpha in zip(list_alpha_mean, list_alpha_covariance, list_alpha_model(*args)):
        # Fill in
        #toret += ...
    return toret

init = [100., 0.3, 0.]  # rs_drag [Mpc/h], Omega_m, Omega_k
bounds = [(20, 1000), (0.01, 0.9), (-0.8, 0.8)]

def logprior(args):
    """Write flat uniform prior between bounds."""

def logposterior(args):
    """Write log-posterior."""

#ndim = len(init)
#nwalkers = 4 * ndim
#nsteps = 2000
#sampler = emcee.EnsembleSampler(nwalkers, ndim, logposterior)
#start = [[rng.normal(v, (b[1] - b[0]) / 40.) for v, b in zip(init, bounds)] for i in range(nwalkers)]
#sampler.run_mcmc(start, nsteps)
#samples = sampler.get_chain(flat=True)
# Remove burnin and plot

### Inverse distance ladder
Do the same, but for $r_{\mathrm{drag}}$ use a Gaussian prior $r_{\mathrm{drag}} = 149.47 \pm 0.48 \; \mathrm{Mpc}$.  
Assume zero curvature for this exercise (though you can let it free in general, in addition to different dark energy models --- all those are constrained by BAO).
Sample the posterior and draw the $\Omega_{m} - H_{0}$ contours. See? given $r_{\mathrm{drag}}$ we constrain $H_{0}$!

In [ ]:
# We just need to redefine the model as a function of rs_drag in Mpc, H0 and Omega_m,
# and the Gaussian prior on rs_drag

def list_alpha_model(rs, H0, *args):
    return np.array([spherically_averaged_distance(z, *args) for z in list_z]) / (H0 / 100.) / rs / list_dv_over_rs_fid

init = [149.47, 70, 0.3]  # rs_drag, H0, Omega_m 
bounds = [(100, 200), (50, 100), (0.1, 0.5)]

def logprior(args):
    """Write flat uniform (log-)prior between bounds, and Gaussian prior on rs_drag."""

#ndim = len(init)
#nwalkers = 4 * ndim
#nsteps = 3000
#sampler = emcee.EnsembleSampler(nwalkers, ndim, logposterior)
#start = [[rng.normal(v, (b[1] - b[0]) / 10.) for v, b in zip(init, bounds)] for i in range(nwalkers)]
#sampler.run_mcmc(start, nsteps)
#samples = sampler.get_chain(flat=True)
# Remove burnin and plot

### BBN prior
CMB constraints provide a measurement of $r_{\mathrm{drag}}$ (through $\omega_{cdm}$ and $\omega_{b}$),
but we only need a prior on $\omega_{b} = \Omega_{b} h^{2}$ since $\Omega_{m}$ is constrained by BAO mesurements.  $\omega_{b} = \Omega_{b} h^{2}$ is constrained by measurements of deuterium abundance as a result of BBN.

In [3]:
# To compute rs_drag, either:
# 1) use cosmoprimo (engine='class')
#def rs_drag(omega_b, omega_m):
#    return cosmo_fid.clone(omega_b=omega_b, omega_m=omega_m, engine='class').rs_drag / cosmo_fid.h
# 2) use cosmoprimo (engine='eisentein_hu') --- faster
#def rs_drag(omega_b, omega_m):
#    return cosmo_fid.clone(omega_b=omega_b, omega_m=omega_m, engine='eisenstein_hu').rs_drag / cosmo_fid.h
# 3) implement rs_drag yourself, following https://arxiv.org/abs/astro-ph/9709112, eq. 4 - 6.
# For $z_{\mathrm{drag}}$, rather use https://arxiv.org/abs/astro-ph/9510117, eq. E1 (found better match to Boltzmann code).
# Fix $T_\mathrm{cmb} = 2.7255$ (Cobe-FIRAS).
#def rs_drag(omega_b, omega_m):
#    """Write approximation for rs_drag as a function of omega_b, omega_m"""

Redo the previous sampling, this time varying $H_{0}$, $\Omega_{m}$ and $\omega_{b}$ with the BBN prior: $\omega_{b} = 0.02235 \pm 0.00037$.

In [4]:
# We just need to redefine the model as a function of omega_b, H0 and Omega_m, and the Gaussian prior on omega_b

def list_alpha_model(omega_b, H0, Omega_m):
    h = H0 / 100.
    rs = rs_drag(omega_b, Omega_m * h**2)
    return np.array([spherically_averaged_distance(z, Omega_m) for z in list_z]) / h / rs / list_dv_over_rs_fid

# For omega_b, H0, Omega_m
init = [0.02235, 70, 0.3]  # omega_b, H0, Omega_m
bounds = [(0.01, 0.03), (50, 100), (0.1, 0.5)]

def logprior(args):
    """Write flat uniform (log-)prior between bounds, and Gaussian prior on omega_b."""

#ndim = len(init)
#nwalkers = 4 * ndim
#nsteps = 3000
#sampler = emcee.EnsembleSampler(nwalkers, ndim, logposterior)
#start = [[rng.normal(v, (b[1] - b[0]) / 10.) for v, b in zip(init, bounds)] for i in range(nwalkers)]
#sampler.run_mcmc(start, nsteps)
#samples = sampler.get_chain(flat=True)
# Remove burnin and plot

## Take-home messages
- The BAO feature is a standard ruler: measuring the position of the BAO peak = measuring a fixed comoving distance (sound horizon at the drag epoch) at a given redshift = constraing the Universe's expansion
- assuming standard ruler only (of unspecified length), we can constrain the 'derivatives of the expansion', i.e. the matter / dark energy density
- with an additional prior on $r_{\mathrm{drag}}$ (or $\omega_{b}$), we can measure $H_{0}$ = 'inverse distance ladder'

## Bonus
- see how $\Omega_{m} - H_{0}$ contour rotates with redshift
- perform fits to real eBOSS data (see eBOSS_LRGpCMASS.ipynb for power spectrum measurements): isotropic, anisotropic BAO fits, RSD fits, ...
- why are you still here? go to the beach...